In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import gseapy as gp
import matplotlib.pyplot as plt
import swan_vis as swan
import yaml
from snakemake.io import expand
import itertools

p = os.path.dirname(os.path.dirname(os.getcwd()))
sys.path.append(p)

from scripts.utils import *
from scripts.plotting import *

In [2]:
config_file = '../snakemake/config.yml'
with open(config_file) as f:
    config = yaml.safe_load(f)

In [3]:
filt_ab = '../'+expand(config['data']['filt_ab'], species='human')[0]
lib_meta = '../'+expand(config['data']['meta'], species='human')[0]
cerberus_h5 = '../'+expand(config['data']['cerb_annot'], species='human')[0]
ca_plus = '../'+expand(config['data']['ca_plus'], species='human')[0]
swan_file = '../'+expand(config['data']['sg'], species='human')[0]
gtf = '../'+expand(config['data']['cerb_gtf'], species='human')[0]

phastcons = '../'+config['ref']['phastcons100']['txt']

gene_subset = 'polya'

biosamp_name_map = '../'+expand(config['ref']['biosamp_map'])[0]

cage_meta = '../'+expand(config['cage']['meta'], species='human')[0]
rampage_meta = '../'+expand(config['rampage']['meta'], species='human')[0]

min_tpm = 1
gene_subset = 'protein_coding'

In [4]:
filt_ab = '../'+expand(config['data']['filt_ab'], species='human')[0]
lib_meta = '../'+expand(config['data']['meta'], species='human')[0]
cerberus_h5 = '../'+expand(config['data']['cerb_annot'], species='human')[0]
ca_plus = '../'+expand(config['data']['ca_plus'], species='human')[0]
swan_file = '../'+expand(config['data']['sg'], species='human')[0]
gtf = '../'+expand(config['data']['cerb_gtf'], species='human')[0]

phastcons = '../'+config['ref']['phastcons100']['txt']

gene_subset = 'polya'

biosamp_name_map = '../'+expand(config['ref']['biosamp_map'])[0]

cage_meta = '../'+expand(config['cage']['meta'], species='human')[0]
rampage_meta = '../'+expand(config['rampage']['meta'], species='human')[0]

min_tpm = 1
gene_subset = 'protein_coding'

In [5]:
gtf_df = pr.read_gtf(gtf, rename_attr=True, duplicate_attr=True)

Found attributes with reserved names: ['source'].
Renaming attributes with suffix '_attr'


In [6]:
gene_df = gtf_df.df
gene_df = gene_df.loc[gene_df.Feature=='gene']
gene_df = gene_df[['gene_id', 'gene_name']]

In [12]:
## mean intron length
introns = gtf_df.features.introns(by='transcript').df
introns['len'] = (introns.Start-introns.End).abs()
temp = introns[['gene_id', 'transcript_id', 'len']].groupby(['gene_id', 'transcript_id']).sum().reset_index()
temp = temp[['gene_id', 'len']].groupby('gene_id').mean().reset_index().rename({'len':'mean_intron_len'}, axis=1)
temp = cerberus.add_stable_gid(temp)
gene_df = gene_df.merge(temp, how='left', on='gene_id')

In [19]:
## mean # introns
temp = introns[['gene_id', 'transcript_id', 'len']].groupby(['gene_id', 'transcript_id']).count().reset_index()
temp = temp[['gene_id', 'len']].groupby('gene_id').mean().reset_index().rename({'len':'mean_n_intron'}, axis=1)
temp = cerberus.add_stable_gid(temp)
gene_df = gene_df.merge(temp, how='left', on='gene_id')

In [20]:
gene_df.head()

,gene_id,gene_name,mean_intron_len,mean_n_intron
0,ENSG00000000460.16,C1orf112,NaN,NaN
1,ENSG00000000971.15,CFH,NaN,NaN
2,ENSG00000001461.16,NIPAL3,NaN,NaN
3,ENSG00000004487.16,KDM1A,NaN,NaN
4,ENSG00000007933.12,FMO3,NaN,NaN


In [ ]:
temp.he